<a href="https://colab.research.google.com/github/as2092/Intent-Classification/blob/master/intent_classification_and_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intent Classification and NER on Chatbot data


In [0]:
import json

In [0]:
with open("ChatbotCorpus.json") as f:
  data = json.load(f)

In [0]:
data

{'lang': 'en',
 'name': 'ChatSample',
 'sentences': [{'entities': [{'entity': 'StationDest',
     'start': 4,
     'stop': 4,
     'text': 'marienplatz'}],
   'intent': 'FindConnection',
   'text': 'i want to go marienplatz',
   'training': False},
  {'entities': [{'entity': 'Vehicle', 'start': 4, 'stop': 4, 'text': 'train'},
    {'entity': 'Criterion', 'start': 3, 'stop': 3, 'text': 'next'},
    {'entity': 'StationStart',
     'start': 6,
     'stop': 7,
     'text': 'muncher freiheit'}],
   'intent': 'DepartureTime',
   'text': 'when is the next train in muncher freiheit?',
   'training': False},
  {'entities': [{'entity': 'Vehicle', 'start': 4, 'stop': 6, 'text': 'u-bahn'},
    {'entity': 'Criterion', 'start': 3, 'stop': 3, 'text': 'next'},
    {'entity': 'StationStart',
     'start': 9,
     'stop': 10,
     'text': 'garching forschungszentrum'}],
   'intent': 'DepartureTime',
   'text': 'when does the next u-bahn leaves from garching forschungszentrum?',
   'training': False},
  {

In [0]:
for d in data['sentences']:
  
data['sentences']

206

# Spacy

In [0]:
!pip install textacy

     |████████████████████████████████| 204kB 6.3MB/s 
     |████████████████████████████████| 3.0MB 39.4MB/s 
     |████████████████████████████████| 481kB 42.4MB/s 
     |████████████████████████████████| 143kB 41.2MB/s 
  Created wheel for cytoolz: filename=cytoolz-0.10.1-cp36-cp36m-linux_x86_64.whl size=1256657 sha256=9980873215c18a019eb15715036dc5e124c58bb1ac6a9d53b4877f61d8be8081
  Stored in directory: /root/.cache/pip/wheels/64/2a/18/d962b614e055577e7d9a3e4813e0742f822ca9c8800cc3783a
  Created wheel for jellyfish: filename=jellyfish-0.7.2-cp36-cp36m-linux_x86_64.whl size=72998 sha256=3f82f46bcabaf385aa8de32542c2628095f66276ea1b8fd8e8aa446c10cd7804
  Stored in directory: /root/.cache/pip/wheels/e8/fe/99/d8fa8f2ef7b82a625b0b77a84d319b0b50693659823c4effb4
Successfully built cytoolz jellyfish


In [0]:
import spacy
from spacy import displacy
import textacy

In [0]:
!python3 -m spacy download en_core_web_lg

     |████████████████████████████████| 826.9MB 48.9MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.1.0-cp36-none-any.whl size=828255076 sha256=94f625d38f443b574705b30f628507fb5cfe91d081399f5cafed344e2bc22352
  Stored in directory: /tmp/pip-ephem-wheel-cache-w41s3n5b/wheels/b4/d7/70/426d313a459f82ed5e06cc36a50e2bb2f0ec5cb31d8e0bdf09
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
import spacy.cli
spacy.cli.download("en_core_web_lg")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
nlp = spacy.load('en_core_web_lg')
spacy.__version__

'2.1.9'

In [0]:
# using displacy to have a look at dependency parsing (NER is also available)
displacy.render(doc, style='dep', jupyter=True)

In [0]:
doc = nlp("Gurdeep is a doctor")
displacy.render(doc, style='ent', jupyter=True)

In [0]:
doc = nlp("ultimate product! i got the results after using it for 2 months")
for e in doc.ents:
  print(e, e.label_)

2 months DATE


# intent classification using  sklearn models

In [0]:
%matplotlib inline

In [0]:
import pandas as pd
import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt

from pandas.io.json import json_normalize  
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression as LR
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

import json

# Reading Data

In [0]:
def get_train_test_data(filename):
  """
  get test and train samples for intent classification from ChatbotCorpus.json
  """
  with open("ChatbotCorpus.json", "r") as f:
    d = json.load(f)
  
  sent_df = json_normalize(d['sentences'])
  sent_df = sent_df[['text', 'intent', 'training']]
  
  intent_train = sent_df[sent_df['training'] == False]
  intent_test = sent_df[sent_df['training'] == True].reset_index()

  intent_train.to_csv('intent_train.csv', columns=['text', 'intent'])
  intent_test.to_csv('intent_test.csv', columns=['text', 'intent'])
  sent_df.to_csv('intent_all.csv', columns=['text', 'intent'])

# generating csv for test and train data
get_train_test_data('ChatbotCorpus.json')

In [0]:
def read_train_test_data(file1, file2):
  """
  read the csv files to dataframes
  returns: dataframes
  """
  train_data = pd.read_csv(file1, usecols=['text', 'intent'])
  test_data = pd.read_csv(file2, usecols=['text', 'intent'])

  return train_data, test_data

# getting train and test data as dataframes
train_data, test_data = read_train_test_data("intent_train.csv", "intent_test.csv")

In [0]:
seq_list = []
for t in train_data['text'].tolist():
  seq_list.extend(list(t))

chars_train = list(set(seq_list))
print(f'no. of chars in train samples: {len(chars_train)}, all chars: {chars_train}')

no. of chars in train samples: 40, all chars: ['u', 'ö', 'ä', 'i', '6', 'y', 'x', 'a', '.', 'm', 'q', ':', 'z', 'n', 'b', 'l', ',', 'v', 't', 'c', 'k', "'", '!', 'g', 'p', 's', '1', 'ü', ' ', 'd', 'r', 'e', '?', 'f', 'ß', 'w', '2', 'h', 'o', '-']


In [0]:
seq_list = []
for t in test_data['text'].tolist():
  seq_list.extend(list(t))

chars_test = list(set(seq_list))
print(f'no. of chars in test samples: {len(chars_test)}, all chars: {chars_test}')

no. of chars in test samples: 43, all chars: ['u', 'ö', 'ä', 'i', '6', 'y', 'a', 'x', '.', 'm', 'q', 'z', 'n', '9', 'b', 'l', ',', 'v', 't', 'c', "'", 'k', '!', 'g', 'p', 's', '8', 'ü', ' ', '1', 'd', 'r', 'e', '?', 'f', '5', '3', 'ß', 'w', '2', 'h', 'o', '-']


In [0]:
# 
len(set(chars_test + chars_train))

44

In [0]:
train_data["intent"].value_counts()

FindConnection    71
DepartureTime     35
Name: intent, dtype: int64

In [0]:
test_data["intent"].value_counts()

FindConnection    57
DepartureTime     43
Name: intent, dtype: int64

# Logistic Regression for intent classification

In [0]:
def get_features(data):
  """
  """
  # creating vocab using CountVectorizer
  count_vect = CountVectorizer()
  x_train_count = count_vect.fit_transform(train_data.text)
  print(f'Term Freq. matrix shape: {x_train_count.shape}')

  # creating features
  tf_idf_transformer = TfidfTransformer()
  x_train_tfidf = tf_idf_transformer.fit_transform(x_train_count)
  print(f'TFIDF matrix shape: {x_train_tfidf.shape}')

  return x_train_count, x_train_tfidf

def train_clf(data, classifier):
  """
  """

  # instantiating the sklearn pipeline
  clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                   ('clf', classifier)])
  
  # training classifier
  clf = clf.fit(train_data.text, train_data.intent)

  return clf


def accuracy_calc(trained_model):
  """
  """
  pred = trained_model.predict(test_data.text)
  clf_acc = np.mean(pred == test_data.intent) * 100
  print(f'test accuracy: {clf_acc}')

  return pred



In [0]:
x_train_count, x_train_tfidf = get_features(train_data)
lr_clf = train_clf(train_data, LR(solver='lbfgs'))  # changing solver from liblinear to lbfgs gives 1% inrease in accuracy
pred_lr = accuracy_calc(lr_clf)

Term Freq. matrix shape: (106, 113)
TFIDF matrix shape: (106, 113)
test accuracy: 96.0


In [0]:
conf_mat = confusion_matrix(y_true=test_data.intent, y_pred = pred_lr)
print(f'confusion matrix: {conf_mat}')

confusion matrix: [[42  1]
 [ 3 54]]


In [0]:
print(classification_report(test_data.intent, pred_lr))

                precision    recall  f1-score   support

 DepartureTime       0.93      0.98      0.95        43
FindConnection       0.98      0.95      0.96        57

      accuracy                           0.96       100
     macro avg       0.96      0.96      0.96       100
  weighted avg       0.96      0.96      0.96       100



# randomForest classifier

In [0]:
RF_clf = train_clf(train_data, RandomForestClassifier()) 
pred_RF = accuracy_calc(RF_clf)

# RF_clf.predict(['how can i reach munich?'])

test accuracy: 94.0


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [0]:
conf_mat = confusion_matrix(y_true=test_data.intent, y_pred = pred_RF)
print(f'confusion matrix: {conf_mat}')

confusion matrix: [[43  0]
 [ 6 51]]


In [0]:
print(classification_report(test_data.intent, pred_RF))

                precision    recall  f1-score   support

 DepartureTime       0.88      1.00      0.93        43
FindConnection       1.00      0.89      0.94        57

      accuracy                           0.94       100
     macro avg       0.94      0.95      0.94       100
  weighted avg       0.95      0.94      0.94       100



In [0]:
# trying grid search
parameters_RF={ "n_estimators" : [50,60,70],
           "min_samples_leaf" : [1, 11]}
gs_RF = train_clf(train_data, GridSearchCV(RandomForestClassifier(n_estimators=8),parameters_RF, cv=3))
pred_gsRF = accuracy_calc(gs_RF)

test accuracy: 96.0


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [0]:
print(classification_report(test_data.intent, pred_gsRF))

                precision    recall  f1-score   support

 DepartureTime       0.91      1.00      0.96        43
FindConnection       1.00      0.93      0.96        57

      accuracy                           0.96       100
     macro avg       0.96      0.96      0.96       100
  weighted avg       0.96      0.96      0.96       100



# SVM

In [0]:
svc_clf = train_clf(train_data, SVC(kernel="linear"))
pred_svc = accuracy_calc(svc_clf)

print(classification_report(test_data.intent, pred_svc))


test accuracy: 94.0
                precision    recall  f1-score   support

 DepartureTime       0.88      1.00      0.93        43
FindConnection       1.00      0.89      0.94        57

      accuracy                           0.94       100
     macro avg       0.94      0.95      0.94       100
  weighted avg       0.95      0.94      0.94       100



In [0]:
parameters = {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}
gs_svc = train_clf(train_data, (GridSearchCV(SVC(), parameters, cv=3)))
pred_gsvc = accuracy_calc(gs_svc)

test accuracy: 96.0


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [0]:
print(classification_report(test_data.intent, pred_gsvc))

                precision    recall  f1-score   support

 DepartureTime       0.91      1.00      0.96        43
FindConnection       1.00      0.93      0.96        57

      accuracy                           0.96       100
     macro avg       0.96      0.96      0.96       100
  weighted avg       0.96      0.96      0.96       100



# SGD Classifier

In [0]:
txt_clf = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('clf', SGDClassifier(loss='hinge', penalty='l2',
                       alpha=1e-3, random_state=42, max_iter=5, tol=None))])

In [0]:
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3), }

In [0]:
gs_clf = GridSearchCV(txt_clf, parameters, cv=5, iid=False, n_jobs=-1)

In [0]:
gs_clf = gs_clf.fit(train_data.text, train_data.intent)

In [0]:
#testing
pred_g = gs_clf.predict(test_data.text)
np.mean(pred_g == test_data.intent)

0.99

In [0]:
confusion_matrix(y_true=test_data.intent, y_pred=pred_g)

array([[42,  1],
       [ 0, 57]])

In [0]:
print(classification_report(test_data.intent, gs_clf))

In [0]:
for param_name in sorted(parameters.keys()):
  print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))


clf__alpha: 0.001
tfidf__use_idf: False
vect__ngram_range: (1, 1)


In [0]:
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3), }
gs_svc = train_clf(train_data, (GridSearchCV(SGDClassifier(), parameters, cv=3)))
pred_gsvc = accuracy_calc(gs_svc)

# NER and intent classification (using rasa_nlu)

In [0]:
!pip install rasa_nlu

In [0]:
!pip install sklearn_crfsuite

Creating data in training format of rasa_nlu

In [0]:
with open("ChatbotCorpus.json", "r") as f:
   d = json.load(f)
  
sent_df = json_normalize(d['sentences'])

In [0]:
sent_df = sent_df[['text', 'entities', 'intent']]

In [0]:
sent_df

,text,entities,intent
0,i want to go marienplatz,"[{'entity': 'StationDest', 'start': 4, 'stop':...",FindConnection
1,when is the next train in muncher freiheit?,"[{'entity': 'Vehicle', 'start': 4, 'stop': 4, ...",DepartureTime
2,when does the next u-bahn leaves from garching...,"[{'entity': 'Vehicle', 'start': 4, 'stop': 6, ...",DepartureTime
3,from olympia einkaufszentrum to hauptbahnhof,"[{'entity': 'StationStart', 'start': 1, 'stop'...",FindConnection
4,when is the next train from winterstraße 12 to...,"[{'entity': 'Vehicle', 'start': 4, 'stop': 4, ...",FindConnection
...,...,...,...
201,connection from hauptbahnhof to odeonsplatz?,"[{'entity': 'StationStart', 'start': 2, 'stop'...",FindConnection
202,how do i get from olympia einkaufszentrum to h...,"[{'entity': 'StationStart', 'start': 5, 'stop'...",FindConnection
203,when is the next bus in garching forschungszen...,"[{'entity': 'Vehicle', 'start': 4, 'stop': 4, ...",DepartureTime
204,how can i get from garching to marienplatz,"[{'entity': 'StationStart', 'start': 5, 'stop'...",FindConnection


In [0]:
sent_df.to_json('ner_train.json', orient='records')

In [0]:
sent_df.iloc[2]['entities']

[{'entity': 'Vehicle', 'start': 4, 'stop': 6, 'text': 'u-bahn'},
 {'entity': 'Criterion', 'start': 3, 'stop': 3, 'text': 'next'},
 {'entity': 'StationStart',
  'start': 9,
  'stop': 10,
  'text': 'garching forschungszentrum'}]

In [0]:
import json
with open('ner_train.json', 'r') as f:
  n = json.load(f)

In [0]:
sent_df.to_csv('ner.md')

In [0]:
nlp = spacy.load('en_core_web_lg')

In [0]:
# getting data in rasa_nlu format
# tokens_1 = []
final_list = []
intent_list = []
for i, row in enumerate(sent_df['text'].tolist()):
  doc = nlp(row)
  tokens = [t.text for t in doc]
  ent_list = sent_df['entities'].tolist()
  int_list = sent_df['intent'].tolist()
  for v in ent_list[i]:
    if v['start'] == v['stop']:
      tokens[v['start']] = "[" + tokens[v['start']] + "](" + v['entity'] + ")"
    else:
      tokens[v['start']] = "[" + tokens[v['start']]
      tokens[v['stop']] = tokens[v['stop']] + "](" + v['entity'] + ")"
  
  final_list.append("- "+" ".join(tokens))
  intent_list.append("## intent:" + int_list[i])
  print(tokens)


In [0]:
with open('ner_train.txt', 'w') as f:
    for item in list(zip(intent_list, final_list)):
        f.write("%s\n" % item[0])
        f.write("%s\n" % item[1])


In [0]:
from rasa_nlu.convert import convert_training_data
convert_training_data(data_file="ner_train.txt", out_file="ner_train.md", output_format="md", language="")

In [0]:
config = """
language: "en"

pipeline:
- name: "nlp_spacy"                   # loads the spacy language model
- name: "tokenizer_spacy"             # splits the sentence into tokens
- name: "ner_crf"                     # uses the pretrained spacy NER model
- name: "intent_featurizer_spacy"     # transform the sentence into a vector representation
- name: "intent_classifier_sklearn"   # uses the vector representation to classify using SVM
- name: "ner_synonyms"                # trains the synonyms
""" 

%store config > config.yml

Writing 'config' (str) to file 'config.yml'.


In [0]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# loading the nlu training samples
training_data = load_data("ner_train.md")

# trainer to educate pipeline
trainer = Trainer(config.load("config.yml"))

# train the model (using entire dataset without train-test split)
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("nlu", fixed_model_name="current")

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.2s finished


In [0]:
interpreter.parse("when does the train leaving in garching forschungszentrum")

{'entities': [{'confidence': 0.9333201273775898,
   'end': 19,
   'entity': 'Vehicle',
   'extractor': 'CRFEntityExtractor',
   'start': 14,
   'value': 'train'},
  {'confidence': 0.9646259709851274,
   'end': 57,
   'entity': 'StationStart',
   'extractor': 'CRFEntityExtractor',
   'start': 31,
   'value': 'garching forschungszentrum'}],
 'intent': {'confidence': 0.985448604199821, 'name': 'DepartureTime'},
 'intent_ranking': [{'confidence': 0.985448604199821, 'name': 'DepartureTime'},
  {'confidence': 0.01455139580017882, 'name': 'FindConnection'}],
 'text': 'when does the train leaving in garching forschungszentrum'}

In [0]:
from rasa_nlu.test import run_evaluation
run_evaluation("ner_train.md", model_directory)

100%|██████████| 206/206 [00:01<00:00, 147.93it/s]


{'entity_evaluation': {'CRFEntityExtractor': {'accuracy': 0.991448118586089,
   'f1_score': 0.99145441855131,
   'precision': 0.991479376466795,
   'report': '               precision    recall  f1-score   support\n\n    Criterion       0.95      0.96      0.96        84\n         Line       1.00      1.00      1.00         3\n  StationDest       0.99      0.98      0.99       152\n StationStart       0.99      1.00      0.99       274\n  TimeEndTime       1.00      1.00      1.00         3\nTimeStartTime       1.00      1.00      1.00         9\n      Vehicle       0.99      1.00      0.99        99\n    no_entity       0.99      0.99      0.99      1130\n\n    micro avg       0.99      0.99      0.99      1754\n    macro avg       0.99      0.99      0.99      1754\n weighted avg       0.99      0.99      0.99      1754\n'}},
 'intent_evaluation': {'accuracy': 1.0,
  'f1_score': 1.0,
  'precision': 1.0,
  'predictions': [{'confidence': 0.9573737954078507,
    'intent': 'DepartureTime

# Other techniques that can be used <br>


*   Deep learning based techniques if we have more data, e.g. BERT + Classification layers for intent as well as ner
*  Data augmentation might be helpful

*   If there are 100k untagged samples, ULMFit can be used for intent classification as the untagged samples can be used for fine tuning the pr--trained Language Model





